<a href="https://colab.research.google.com/github/adithya7shankar/VNL-CSE-597-A7S/blob/main/VNLHW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving todo.py to todo.py
Saving dataset.pkl to dataset.pkl
Saving dataset.py to dataset.py
Saving lr_warmup.py to lr_warmup.py
Saving main.ipynb to main.ipynb
Saving requirements.txt to requirements (1).txt
Saving trainer.py to trainer.py
Saving utils.py to utils.py


In [ ]:
pip install -r requirements.txt

In [ ]:
#!python3 "/main.ipynb"

Traceback (most recent call last):
  File "/main.ipynb", line 5, in <module>
    "execution_count": null,
NameError: name 'null' is not defined


In [ ]:
import os

import torch
import torch.optim as optim
from torch.utils.data import random_split

from dataset import MiniFlickrDataset, get_loader
from todo import CaptioningModel
from trainer import Trainer
from lr_warmup import LRWarmup
import utils


In [ ]:
class Args:
    def __init__(self):
        self.data_path = 'dataset.pkl'
        self.clip_model = 'openai/clip-vit-base-patch32'
        self.text_model = 'gpt2'
        self.seed = 100
        self.num_workers = 2
        self.train_size = 0.84
        self.val_size = 0.13
        self.test_size = 100
        self.epochs = 150
        self.lr = 3e-3
        self.k = 0.33
        self.batch_size_exp = 6
        self.ep_len = 4
        self.num_layers = 6
        self.n_heads = 16
        self.forward_expansion = 4
        self.max_len = 40
        self.dropout = 0.1
config = Args()


In [ ]:
device = utils.init_env(config.seed)

# Create data loaders
dataset = MiniFlickrDataset(config.data_path)
config.train_size = int(config.train_size * len(dataset))
config.val_size = len(dataset) - config.train_size - config.test_size
train_dataset, val_dataoet, test_dataset = random_split(dataset, [config.train_size, config.val_size, config.test_size])
train_loader = get_loader(
    train_dataset,
    bs_exp=config.batch_size_exp,
    shuffle=True,
    num_workers=config.num_workers,
    pin_memory=True,
)
test_loader = get_loader(
    test_dataset,
    bs_exp=0,
    shuffle=False,
    num_workers=config.num_workers,
    pin_memory=True,
    train=False,
)

# Creat model
model = CaptioningModel(
    clip_model=config.clip_model,
    text_model=config.text_model,
    ep_len=config.ep_len,
    num_layers=config.num_layers,
    n_heads=config.n_heads,
    forward_expansion=config.forward_expansion,
    dropout=config.dropout,
    max_len=config.max_len,
    device=device
)

# Create optimizer, lr scheduler
optimizer = optim.Adam(model.parameters(), lr=config.lr)
warmup = LRWarmup(epochs=config.epochs, max_lr=config.lr, k=config.k)
scheduler = optim.lr_scheduler.LambdaLR(optimizer, warmup.lr_warmup)

# Create trainer
trainer = Trainer(
    model=model,
    optimizer=optimizer,
    scaler=torch.cuda.amp.GradScaler(),
    scheduler=scheduler,
    train_loader=train_loader,
    test_loader=test_loader,
    device=device
)


In [ ]:
# Start training
for epoch in range(trainer.epoch, config.epochs):
    trainer.train_epoch()

    score = trainer.test_epoch()
    print("Score: {:.4f}".format(score))

    os.makedirs("checkpoints", exist_ok=True)
    if (epoch + 1) % 6 == 0:
        trainer.save_ckp(os.path.join("checkpoints", f'epoch_{epoch + 1}.pt'))

Epoch: 1 | Loss: 10.804: 100%|██████████| 2086/2086 [13:24<00:00,  2.59it/s]
100%|██████████| 100/100 [01:02<00:00,  1.61it/s]


Score: 0.0333


Epoch: 2 | Loss: 10.337: 100%|██████████| 2086/2086 [13:31<00:00,  2.57it/s]
100%|██████████| 100/100 [00:18<00:00,  5.50it/s]


Score: 0.0374


Epoch: 3 | Loss: 10.149: 100%|██████████| 2086/2086 [13:27<00:00,  2.58it/s]
100%|██████████| 100/100 [00:03<00:00, 31.30it/s]
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


...

Epoch: 12 | Loss: 10.089: 100%|██████████| 2086/2086 [13:30<00:00,  2.57it/s]
100%|██████████| 100/100 [01:10<00:00,  1.41it/s]


Score: 0.0492


Epoch: 13 | Loss: 10.087: 100%|██████████| 2086/2086 [13:34<00:00,  2.56it/s]
100%|██████████| 100/100 [00:40<00:00,  2.50it/s]


Score: 0.0485


Epoch: 14 | Loss: 10.086:  10%|█         | 211/2086 [01:22<12:59,  2.41it/s]